In [447]:
import numpy as np
from collections import defaultdict
import copy

In [686]:
with open('input12.txt', "r") as file:
        text = file.read().strip().split('\n')
grid = np.array([list(x) for x in text])
# Example usage:
# grid = np.array([list(x) for x in """RRRRIICCFF
# RRRRIICCCF
# VVRRRCCFFF
# VVRCCCJFFF
# VVVVCJJCFE
# VVIVCCJJEE
# VVIIICJJEE
# MIIIIIJJEE
# MIIISIJEEE
# MMMISSJEEE""".strip().split('\n')])
# grid = np.array([list(x) for x in """EEEEE
# EXXXX
# EEEEE
# EXXXX
# EEEEE""".strip().split('\n')])
# grid = np.array([list(x) for x in """AAAA
# BBCD
# BBCC
# EEEC""".strip().split('\n')])
# grid = np.array([list(x) for x in """AAAAAA
# AAABBA
# AAABBA
# ABBAAA
# ABBAAA
# AAAAAA""".strip().split('\n')])
# grid = np.array([list(x) for x in """EE
# EX
# EX
# EE
# EX
# EX
# EE""".strip().split('\n')])
# grid = np.array([list(x) for x in """EEE
# EXE
# EXE
# EEE
# EXE
# EXE
# EEE""".strip().split('\n')])

In [687]:
from collections import deque

def is_valid_move(x, y, grid, visited, start_value):
    rows, cols = len(grid), len(grid[0])
    return 0 <= x < rows and 0 <= y < cols and grid[x][y] == start_value and not visited[x][y]

def bfs(grid, start):
    rows, cols = len(grid), len(grid[0])
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

    queue = deque([(start, [start])])
    visited[start[0]][start[1]] = True
    start_value = grid[start]
    path2 = [start]
    while queue:
        (x, y), path = queue.popleft()

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(nx, ny, grid, visited, start_value):
                path2.append((nx, ny))
                visited[nx][ny] = True
                queue.append(((nx, ny), path + [(nx, ny)]))

    return path2

start = (0, 0)
visited = []
areas_dict = {}
counter = 0
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        start = (i, j)
        if start not in visited:
            path = bfs(grid, start)
            areas_dict[counter] = path
            visited.extend(path)
            counter += 1

In [688]:
cost = 0
perim_coords = defaultdict(list)
for area_key, area in areas_dict.items():
    perim = 0
    for coord in area:
        i, j = coord
        for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
            if (x, y) not in area:
                perim += 1
                perim_coords[area_key].append((x, y))
                
    cost += len(area) * perim
cost

1374934

In [689]:
def find_lines(coord, list_coords, new_line):
    i, j = coord
    new_line.append((i, j))
    new_coords = list_coords.copy()
    new_coords.remove(coord)
    for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
        if (x, y) in list_coords and (x * len(new_line) == sum([x[0] for x in new_line]) or
                                      y * len(new_line) == sum([x[1] for x in new_line])) and (
                                          (x, y) not in new_line
                                      ):
            find_lines((x, y), new_coords, new_line)
    return new_line

lines_of_grid = []
missing_lines = []
test_dict = copy.deepcopy(perim_coords)
for _, perim_coords_c in test_dict.items():
    num_coords = len(perim_coords_c)
    lines = []
    while len(perim_coords_c) > 0:
        coord = perim_coords_c[0]
        line = find_lines(coord, perim_coords_c, [])
        lines.append(line)
        for c in line:
            perim_coords_c.remove(c)
        # HOTFIX
        # pair_lines = [line for line in lines if len(line) == 2]
        # if pair_lines != []:
        #     missing_counts = sum([x if x > 1 else 0 for x in Counter([tuple(x) for x in pair_lines]).values()])
        #     missing_lines.append(len(lines) + missing_counts // 2)
        # else:
        #     missing_lines.append(len(lines))
    lines_of_grid.append(lines)
#missing_lines = [x-1 if x % 2 == 1 else x for x in missing_lines]

In [690]:
lens = [len(x)  for x in lines_of_grid]
areas = [len(x) for x in areas_dict.values()]
print(areas)
print(lens)
dot_product = sum(a * b for a, b in zip(lens, areas))
print(cost, dot_product)
print(missing_lines)
dot_product = sum(a * b for a, b in zip(missing_lines, areas))
print(cost, dot_product)

[121, 1, 9, 1, 1, 124, 22, 27, 35, 73, 104, 134, 1, 26, 70, 8, 68, 4, 142, 2, 57, 1, 37, 66, 4, 1, 12, 73, 102, 1, 51, 1, 6, 1, 2, 1, 1, 42, 1, 64, 20, 68, 2, 26, 27, 128, 35, 19, 6, 125, 130, 44, 3, 30, 114, 1, 1, 3, 1, 1, 1, 57, 88, 2, 135, 142, 223, 2, 1, 1, 59, 55, 1, 1, 84, 14, 1, 1, 2, 2, 145, 1, 1, 1, 93, 1, 52, 31, 72, 49, 10, 35, 1, 18, 2, 1, 1, 1, 1, 130, 4, 1, 184, 9, 23, 142, 107, 6, 11, 79, 8, 5, 145, 1, 5, 9, 100, 6, 2, 27, 21, 16, 1, 1, 11, 5, 59, 27, 1, 192, 10, 113, 3, 1, 12, 1, 1, 1, 1, 39, 1, 25, 1, 102, 3, 102, 5, 116, 3, 1, 66, 39, 4, 2, 158, 32, 14, 4, 1, 85, 8, 1, 102, 1, 32, 7, 1, 1, 103, 6, 75, 89, 3, 1, 130, 2, 5, 1, 105, 1, 120, 5, 60, 2, 1, 70, 6, 185, 1, 1, 2, 25, 131, 18, 1, 3, 2, 281, 2, 87, 26, 122, 2, 1, 1, 1, 1, 1, 1, 94, 2, 76, 1, 8, 83, 1, 1, 2, 107, 56, 1, 12, 22, 3, 2, 63, 1, 1, 1, 1, 117, 4, 167, 38, 1, 1, 39, 1, 1, 133, 1, 2, 45, 1, 2, 99, 3, 2, 4, 113, 1, 1, 68, 138, 19, 1, 111, 1, 1, 61, 2, 51, 42, 3, 2, 13, 41, 1, 5, 1, 27, 1, 1, 3, 8, 19, 1, 

In [685]:
# 825295 too low + 81 too low, 832322
# 335 - the one to fix

In [697]:
R = grid.shape[0]
C = grid.shape[1]
G = open('input12.txt').read().strip().split('\n')
DIRS = [(-1, 0), (0, 1), (1, 0), (0, -1)]
SEEN = set()
ans = 0
for r in range(R):
    for c in range(C):
        if (r, c) in SEEN:
            continue
        Q = deque([(r, c)])
        area = 0
        PERIM = dict()
        while Q:
            r2, c2 = Q.popleft()
            if (r2, c2) in SEEN:
                continue
            SEEN.add((r2, c2))
            area += 1
            for dr, dc in DIRS:
                rr = r2+dr
                cc = c2+dc
                if 0<=rr<R and 0<=cc<C and G[rr][cc]==G[r2][c2]:
                    Q.append((rr, cc))
                else:
                    if (dr, dc) not in PERIM:
                        PERIM[(dr, dc)] = set()
                    PERIM[(dr, dc)].add((r2, c2))
        sides = 0
        for k, vs in PERIM.items():
            SEEN_PERIM = set()
            for pr, pc in vs:
                if (pr, pc) not in SEEN_PERIM:
                    sides += 1
                    Q = deque([(pr, pc)])
                    while Q:
                        r2, c2 = Q.popleft()
                        if (r2, c2) in SEEN_PERIM:
                            continue
                        SEEN_PERIM.add((r2, c2))
                        for dr, dc in DIRS:
                            rr, cc = r2+dr, c2+dc
                            if (rr, cc) in vs:
                                Q.append((rr, cc))
        ans += sides * area
print(ans)

841078
